In [1]:
import numpy as np
import pandas as pd
import random
from collections import Counter

from scipy.sparse import csr_matrix
import scipy.sparse

from lightfm import LightFM
from lightfm.evaluation import precision_at_k,auc_score,reciprocal_rank

C:\Users\mm4me\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [3]:
# load in data
r = pd.read_csv("review_r_and_f", index_col="review_id")
print(r.shape, len(r.index))

b = pd.read_csv("business_r_and_f_8.4.csv", index_col="business_id")
print(b.shape, len(b.index))

(1953249, 8) 1953249
(18504, 202) 18504


In [13]:
u = pd.read_csv("user_r_and_f", index_col="user_id")
print(u.shape, len(u.index))

(654161, 22) 654161


In [4]:
b.head()

,name,city,las_vegas,stars,price,review_count,1star_ratio,2star_ratio,3star_ratio,4star_ratio,...,Thai,Turkish,Vegan,Vegetarian,Venues & Event Spaces,Vietnamese,Waffles,Wine & Spirits,Wine Bars,Wraps
business_id,,,,,,,,,,,,,,,,,,,,,
--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,Las Vegas,1,4.0,4.0,NaN,0.032988,0.088616,0.113842,0.271669,...,0,0,0,0,0,0,0,0,0,0
--DaPTJW3-tB1vP-PfdTEg,Sunnyside Grill,Toronto,0,3.5,2.0,NaN,0.088889,0.066667,0.200000,0.355556,...,0,0,0,0,0,0,0,0,0,0
--KCl2FvVQpvjzmZSPyviA,Hungry Howie's Pizza,Charlotte,0,3.0,1.0,NaN,0.230769,0.153846,0.230769,0.076923,...,0,0,0,0,0,0,0,0,0,0
--cZ6Hhc9F7VkKXxHMVZSQ,Pio Pio,Charlotte,0,4.0,2.0,NaN,0.066066,0.066066,0.129129,0.333333,...,0,0,0,0,0,0,0,0,0,0
-01XupAWZEXbdNbxNg5mEg,18 Degrees Neighborhood Grill,Scottsdale,0,3.0,2.0,NaN,0.243902,0.170732,0.207317,0.268293,...,0,0,0,0,0,0,0,0,0,0


In [5]:
r.head()

,business_id,cool,date,funny,stars,text,useful,user_id
review_id,,,,,,,,
x7mDIiDB3jEiPGPHOmDzyw,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,2,The pizza was okay. Not the best I've had. I p...,0,msQe1u7Z_XuqjGoqhB0J5g
dDl8zu1vWPdKGihJrwQbpw,pomGBqfbxcqPv14c3XH-ZQ,0,2012-11-13,0,5,I love this place! My fiance And I go here atl...,0,msQe1u7Z_XuqjGoqhB0J5g
LZp4UX5zK3e-c5ZGSeo3kA,jtQARsP6P-LbkyjbO1qNGg,1,2014-10-23,1,1,Terrible. Dry corn bread. Rib tips were all fa...,3,msQe1u7Z_XuqjGoqhB0J5g
Er4NBWCmCD4nM8_p1GRdow,elqbBhBfElMNSrjFqW3now,0,2011-02-25,0,2,Back in 2005-2007 this place was my FAVORITE t...,2,msQe1u7Z_XuqjGoqhB0J5g
kUZWBVZvhWuC8TWUg5AYyA,zdE82PiD6wquvjYLyhOJNA,0,2012-04-18,1,5,AMAZING!!!\n\n I was referred here by a friend...,0,msQe1u7Z_XuqjGoqhB0J5g


In [14]:
u.head()

,name,review_count,average_stars,yelping_since,friends,fans,elite,cool,funny,useful,...,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,compliment_profile,compliment_writer,friend_count
user_id,,,,,,,,,,,,,,,,,,,,,
lzlZwIpuSWXEnNS91wxjHw,Susan,1,2.00,2015-09-28,None,0,None,0,0,0,...,0,0,0,0,0,0,0,0,0,0
XvLBr-9smbI0m_a7dXtB7w,Daipayan,2,5.00,2015-09-05,None,0,None,0,0,0,...,0,0,0,0,0,0,0,0,0,0
QPT4Ud4H5sJVr68yXhoWFw,Andy,1,4.00,2016-07-21,None,0,None,0,0,0,...,0,0,0,0,0,0,0,0,0,0
s4FoIXE_LSGviTHBe8dmcg,Shashank,3,3.00,2017-06-18,None,0,None,0,0,0,...,0,0,0,0,0,0,0,0,0,0
nnB0AE1Cxp_0154xkhXelw,Mike,11,2.77,2014-07-11,None,1,None,0,0,8,...,0,0,0,0,0,0,0,0,0,0


In [6]:
r.user_id.nunique()

654161

In [7]:
# convert stars values into positive or negative
r['sentiment'] = r['stars'].apply(lambda x: 1 if x >3 else -1)

### Subset to Las Vegas and Phoenix Only

In [19]:
b_lv = b[b.city.isin(['Las Vegas', 'Phoenix'])]
print(f'Top 5 City Business: {b.shape}')
print(f'Las Vegas and Phoenix Business: {b_lv.shape}')

r_lv = r[r.business_id.isin(b_lv.index)]
print(f'Top 5 City Review: {r.shape}')
print(f'Las Vegas and Phoenix Review: {r_lv.shape}')

u_lv = u[u.index.isin(r_lv.user_id)]
print(f'Top 5 City User: {u.shape}')
print(f'Las Vegas and Phoenix User: {u_lv.shape}')

Top 5 City Business: (18504, 202)
Las Vegas and Phoenix Business: (8329, 202)
Top 5 City Review: (1953249, 9)
Las Vegas and Phoenix Review: (1291335, 9)
Top 5 City User: (654161, 22)
Las Vegas and Phoenix User: (482806, 22)


### Subset business and user features

In [24]:
# subset business columns and reorder
business_features = [
 'stars',
 'price',
 'review_count',
 '1star_ratio',
 '2star_ratio',
 '3star_ratio',
 '4star_ratio',
 '5star_ratio',
 'word_count',
 'text_friendly',
 'text_reasonable',
 'text_local',
 'text_comfortable',
 'text_easy',
 'text_quick',
 'cat_Restaurants',
 'cat_Food',
 'cat_Shopping',
 'cat_Active Life',
 'cat_Arts & Entertainment',
 'cat_Nightlife',
 'cat_Hotels & Travel',
 'cat_Tours',
 'Acai Bowls',
 'Afghan',
 'African',
 'American (New)',
 'American (Traditional)',
 'Arcades',
 'Asian Fusion',
 'Bagels',
 'Bakeries',
 'Barbeque',
 'Bars',
 'Bed & Breakfast',
 'Beer',
 'Beer Bar',
 'Bistros',
 'Brasseries',
 'Brazilian',
 'Breakfast & Brunch',
 'Breweries',
 'British',
 'Bubble Tea',
 'Buffets',
 'Burgers',
 'Butcher',
 'Cafes',
 'Cajun/Creole',
 'Canadian (New)',
 'Candy Stores',
 'Cantonese',
 'Caribbean',
 'Casinos',
 'Caterers',
 'Cheesesteaks',
 'Chicken Shop',
 'Chicken Wings',
 'Chinese',
 'Chocolatiers & Shops',
 'Cocktail Bars',
 'Coffee & Tea',
 'Coffee Roasteries',
 'Comfort Food',
 'Convenience Stores',
 'Creperies',
 'Cupcakes',
 'Custom Cakes',
 'Dance Clubs',
 'Delicatessen',
 'Delis',
 'Department Stores',
 'Desserts',
 'Dim Sum',
 'Diners',
 'Dive Bars',
 'Do-It-Yourself Food',
 'Donairs',
 'Donuts',
 'Drugstores',
 'Ethiopian',
 'Ethnic Food',
 'Event Planning & Services',
 'Falafel',
 'Farmers Market',
 'Fashion',
 'Fast Food',
 'Filipino',
 'Fish & Chips',
 'Flowers & Gifts',
 'Food Court',
 'Food Delivery Services',
 'Food Stands',
 'Food Trucks',
 'French',
 'Fruits & Veggies',
 'Gastropubs',
 'Gelato',
 'German',
 'Gluten-Free',
 'Golf',
 'Greek',
 'Grocery',
 'Halal',
 'Hawaiian',
 'Health Markets',
 'Home & Garden',
 'Hookah Bars',
 'Hot Dogs',
 'Hot Pot',
 'Hotels',
 'Ice Cream & Frozen Yogurt',
 'Imported Food',
 'Indian',
 'International Grocery',
 'Internet Cafes',
 'Irish',
 'Irish Pub',
 'Italian',
 'Japanese',
 'Juice Bars & Smoothies',
 'Karaoke',
 'Kebab',
 'Korean',
 'Kosher',
 'Latin American',
 'Lebanese',
 'Local Flavor',
 'Lounges',
 'Meat Shops',
 'Mediterranean',
 'Mexican',
 'Middle Eastern',
 'Modern European',
 'Music Venues',
 'Noodles',
 'Organic Stores',
 'Pakistani',
 'Pan Asian',
 'Party & Event Planning',
 'Pasta Shops',
 'Patisserie/Cake Shop',
 'Persian/Iranian',
 'Peruvian',
 'Pizza',
 'Poke',
 'Polish',
 'Pool Halls',
 'Portuguese',
 'Poutineries',
 'Pubs',
 'Ramen',
 'Salad',
 'Salvadoran',
 'Sandwiches',
 'Seafood',
 'Seafood Markets',
 'Shaved Ice',
 'Shopping Centers',
 'Soul Food',
 'Soup',
 'Southern',
 'Spanish',
 'Specialty Food',
 'Sports Bars',
 'Steakhouses',
 'Street Vendors',
 'Sushi Bars',
 'Szechuan',
 'Tacos',
 'Taiwanese',
 'Tapas Bars',
 'Tapas/Small Plates',
 'Tea Rooms',
 'Tex-Mex',
 'Thai',
 'Turkish',
 'Vegan',
 'Vegetarian',
 'Venues & Event Spaces',
 'Vietnamese',
 'Waffles',
 'Wine & Spirits',
 'Wine Bars',
 'Wraps'
]

# separate out excluded columns
business_features_detail = [
    'name',
    'city',
    'las_vegas',
    'state',
    'address',
    'neighborhood',
    'postal_code',
    'latitude',
    'longitude',
    'monday',
    'tuesday',
    'wednesday',
    'thursday',
    'friday',
    'saturday',
    'sunday',
    'text'
]

In [207]:
# columns to return for recommendation output
business_features_reco = [
    'name',
    'stars',
    'price',
]

In [12]:
# subset business columns and reorder
user_features = [
 'review_count',
 'average_stars',
 'yelping_since',
 'friend_count'
 'cool',
 'funny',
 'useful'
                ]

### Las Vegas-Phoenix Intersection

In [62]:
r['city'] = r['business_id'].map(b['city'])

In [63]:
# users in Las Vegas
u_lv = Counter([u for u in r['user_id'][r['city'] == 'Las Vegas']])

# users in Phoenix
u_ph = Counter([u for u in r['user_id'][r['city'] == 'Phoenix']])

In [64]:
# get intersection of users in Las Vegas and Phoenix
u_lv_ph = list(set([u for u in u_lv.keys()]) & set([u for u in u_ph.keys()]))
len(u_lv_ph)

13346

In [66]:
# subset by users in both cities and businesses that have been rated by those users
r_lv_ph = r[(r['user_id'].isin(u_lv_ph)) & (r['city'].str.contains('Las Vegas|Phoenix'))]
print(r_lv_ph.business_id.nunique(), r_lv_ph.user_id.nunique())

6996 13346


In [58]:
b_lv = Counter([b for b in b.index[b.city == 'Las Vegas']])
len(b_lv)

5077

In [67]:
r[r.user_id == '--4q8EyqThydQm-eKZpS-A']

,business_id,cool,date,funny,stars,text,useful,user_id,sentiment,city
review_id,,,,,,,,,,
3tpZ-RUfSw9z91j2mtxcbA,fHM09_y3QX3n4a_bIFbk_w,0,2012-06-17,0,1,"45 minutes and nothing more than bread, water ...",0,--4q8EyqThydQm-eKZpS-A,-1,Las Vegas
a3eK3obDA8z8MTOjcJrfVA,rcaPajgKOJC2vo_l3xa42A,0,2012-06-17,0,3,Good brunch for a pretty penny.\n\nI thought I...,2,--4q8EyqThydQm-eKZpS-A,-1,Las Vegas


In [72]:
r[r.business_id == 'CFyPbqZaxoBTjOmVjSn_8Q']

,business_id,cool,date,funny,stars,text,useful,user_id,sentiment,city
review_id,,,,,,,,,,
pQ7NHetypf147ytZoRu_Mg,CFyPbqZaxoBTjOmVjSn_8Q,0,2015-12-30,0,5,"Thus place may seem small, but their flavor is...",0,eeovuG852zAmPm-8UPLEiQ,1,Phoenix
9yAGWlB_91kauH4gpI_yIA,CFyPbqZaxoBTjOmVjSn_8Q,0,2017-09-07,0,5,We loved Sonora Taco Shop! Went mid afternoon...,0,DUFqrGcBD6oEO2Ea0iCs6w,1,Phoenix
sq4i5FzOfTOsLwl1XQaH-Q,CFyPbqZaxoBTjOmVjSn_8Q,0,2017-08-06,0,5,Authentic Carne Esada Tacos...We were the only...,0,EXjoQJj-H41dglz-Ck2UWA,1,Phoenix
xhtDMCyB849kdJxUZDB6iw,CFyPbqZaxoBTjOmVjSn_8Q,0,2013-07-09,0,5,Best street tacos in the state and sooo close ...,1,m-OpLxewxny8FIQZjUoNOA,1,Phoenix
FgI8aN8TqU68l2o6WYpBsw,CFyPbqZaxoBTjOmVjSn_8Q,0,2015-06-30,2,5,Being a self proclaimed street taco virtuoso a...,1,s7qRpzKianC6cxq1nJWeaA,1,Phoenix
Y8tUyT-dRIUyzejR15bIXg,CFyPbqZaxoBTjOmVjSn_8Q,2,2013-04-12,0,2,Came here on a weekday just after lunch rush. ...,2,zQja95J54KvlWSGLe4jW7A,-1,Phoenix
b9Qv1JbvDRiGP66JLJLeNg,CFyPbqZaxoBTjOmVjSn_8Q,1,2016-12-28,0,5,You won't be disappointed. Excellent carne asa...,1,BZU2D-2RR9eSsydHApLfVw,1,Phoenix
TIKFVtvoj5r-OUJPaBZmqw,CFyPbqZaxoBTjOmVjSn_8Q,0,2016-08-28,0,2,"No chips, burritos are small, salsas aren't ho...",0,XeE-GuVu81Bgv8AqlJU5bA,-1,Phoenix
ouXnl8haxTuJrzxIIF33YQ,CFyPbqZaxoBTjOmVjSn_8Q,0,2014-09-12,0,3,These tacos are pretty good but pricey $$$ and...,0,wdigY-z-wjH1EaTnQXPF7g,-1,Phoenix


#### Building the ID mappings

In [20]:
from lightfm.data import Dataset

dataset = Dataset()
dataset.fit(r['user_id'], r['business_id'])
print(type(dataset))

no_users, no_items = dataset.interactions_shape()
print('Number of users: {}\nNumber of businesses: {}'.format(no_users, no_items))

<class 'lightfm.data.Dataset'>
Number of users: 654161
Number of businesses: 18504


#### Building the interactions matrix

In [23]:
(interactions, weights) = dataset.build_interactions([(x['user_id'],
                                                      x['business_id'],
                                                      x['sentiment']) for ind, x in r.iterrows()])

print(repr(interactions))

<654161x18504 sparse matrix of type '<class 'numpy.int32'>'
	with 1953249 stored elements in COOrdinate format>


#### Add business and user features

In [25]:
dataset.fit_partial(items=b.index, item_features=business_features)
dataset.fit_partial(users=u.index, user_features=user_features)

In [26]:
# check data
print(type(dataset))
print(dataset.model_dimensions())
print(dataset.item_features_shape())
print(dataset.user_features_shape())
print(dataset.interactions_shape())

<class 'lightfm.data.Dataset'>
(654167, 18689)
(18504, 18689)
(654161, 654167)
(654161, 18504)


#### Build business and user features

In [38]:
item_features = dataset.build_item_features((index, business_features) for index, _x in b.iterrows())
#user_features = dataset.build_user_features((index, user_features) for index, _x in u.iterrows())

print(repr(item_features))
#print(repr(user_features))

<18504x18689 sparse matrix of type '<class 'numpy.float32'>'
	with 3441744 stored elements in Compressed Sparse Row format>


#### Building a model

In [39]:
# train-test split
from lightfm.cross_validation import random_train_test_split

seed = 42
train, test = random_train_test_split(interactions,
                                      test_percentage=0.2,
                                      random_state=np.random.RandomState(seed))

print(f'The dataset has {train.shape[0]} users and {train.shape[1]} items, '
      'with {train.getnnz()} interactions in the train and {test.getnnz() interactions in the test set.')

train.multiply(test).nnz == 0 # make sure train and test are truly disjoint

The dataset has 654161 users and 18504 items, with {train.getnnz()} interactions in the train and {test.getnnz() interactions in the test set.


True

In [44]:
# train-test split -- SAMPLE
seed = 42
sample_size = 60000
interactions_sp = interactions.tocsr()[np.random.sample(sample_size), :]

train_sp, test_sp = random_train_test_split(interactions_sp,
                                      test_percentage=0.2,
                                      random_state=np.random.RandomState(seed))

print('The sample dataset has %s users and %s items, '
      'with %s interactions in the training and %s interactions in the test set.'
      % (interactions_sp.shape[0], interactions_sp.shape[1], train_sp.getnnz(), test_sp.getnnz()))

train_sp.multiply(test_sp).nnz == 0 # make sure train and test are truly disjoint

The sample dataset has 60000 users and 18504 items, with 336000 interactions in the training and 84000 interactions in the test set.


True

In [131]:
h_model_warpkos_full = LightFM(loss='warp-kos')
%time h_model_warpkos_full.fit(train, item_features=item_features)

%time h_auc_warpkos_full_train = auc_score(h_model_warpkos_full, train).mean()
%time h_auc_warpkos_full_test = auc_score(h_model_warpkos_full, test).mean()

print('Hybrid CF WARP-KOS (full) train AUC: %.4f' % h_auc_warpkos_full_train)
print('Hybrid CF WARP-KOS (full) test AUC: %.4f' % h_auc_warpkos_full_test)

Wall time: 13.1 s
Wall time: 18min 37s
Wall time: 7min 46s
Hybrid CF WARP-KOS (full) train AUC: 0.8688
Hybrid CF WARP-KOS (full) test AUC: 0.8624


#### CF and Hybrid - Logistic, WARP

In [114]:
h_model_warpkos = LightFM(loss='warp-kos')
%time h_model_warpkos.fit(train, item_features=item_features)

print('Hybrid CF train AUC: %.4f' % auc_score(h_model_warpkos, train).mean())
print('Hybrid CF test AUC: %.4f' % auc_score(h_model_warpkos, test).mean())

Wall time: 9.68 s
Hybrid CF train AUC: 0.8700
Hybrid CF test AUC: 0.8637


In [130]:
h_model_warp_full = LightFM(loss='warp')
%time h_model_warp_full.fit(train, item_features=item_features)

%time h_auc_warp_full_train = auc_score(h_model_warp_full, train).mean()
%time h_auc_warp_full_test = auc_score(h_model_warp_full, test).mean()

print('Hybrid CF train AUC: %.4f' % h_auc_warp_full_train)
print('Hybrid CF test AUC: %.4f' % h_auc_warp_full_test)

Wall time: 12.3 s
Wall time: 17min 45s
Wall time: 7min 4s
Hybrid CF train AUC: 0.8705
Hybrid CF test AUC: 0.8651


In [113]:
h_model_warp = LightFM(loss='warp')
%time h_model_warp.fit(train, item_features=item_features)

print('Hybrid CF train AUC: %.4f' % auc_score(h_model_warp, train).mean())
print('Hybrid CF test AUC: %.4f' % auc_score(h_model_warp, test).mean())

Wall time: 9.63 s
Hybrid CF train AUC: 0.8717
Hybrid CF test AUC: 0.8663


In [115]:
h_model_bpr = LightFM(loss='bpr')
%time h_model_bpr.fit(train, item_features=item_features)

%time print('Hybrid CF train AUC: %.4f' % auc_score(h_model_bpr, train).mean())
%time print('Hybrid CF test AUC: %.4f' % auc_score(h_model_bpr, test).mean())

Wall time: 8.33 s
Hybrid CF train AUC: 0.5221
Hybrid CF test AUC: 0.5209


In [129]:
h_model_logistic = LightFM(loss='logistic')
%time h_model_logistic.fit(train, item_features=item_features)

%time h_auc_logistic_train = auc_score(h_model_logistic, train).mean()
%time h_auc_logistic_test = auc_score(h_model_logistic, test).mean()

print('Hybrid CF train AUC: %.4f' % h_auc_logistic_train)
print('Hybrid CF test AUC: %.4f' % h_auc_logistic_test)

Wall time: 5.99 s
Wall time: 18min 30s
Wall time: 7min 22s
Hybrid CF train AUC: 0.8618
Hybrid CF test AUC: 0.8560


In [20]:
# loop around different loss functions with SAMPLE

'''
Logistic:
Useful when both positive and negative interactions are present.

BPR: Bayesian Personalised Ranking:
Maximises the prediction difference between a positive example and a randomly chosen negative example

WARP: Weighted Approximate-Rank Pairwise:
Maximises the rank of positive examples by repeatedly sampling negative examples until
rank violation is found. Useful when only positive ineractions are present and optimising
the top of the reommendation list is desired.

k-OS WARP: k-th order statistic loss:
A modification to WARP that uses k-th positive example for any given user as a basis for pairwise updates
'''

loss_func = ['logistic', 'bpr', 'warp', 'warp-kos']

for loss in loss_func:
    model = LightFM(loss=loss)
    model.fit(train, item_features=item_features_T)
    
    %time print(f'Hybrid_{loss} train AUC: {auc_score(model, train).mean():.4f}')
    print(f'Hybrid_{loss} test AUC {auc_score(model, test).mean():.4f}')
    print('\n')

Hybrid_logistic train AUC: 0.8571
Wall time: 11min 41s
Hybrid_logistic test AUC 0.8527


Hybrid_bpr train AUC: 0.5196
Wall time: 14min 21s
Hybrid_bpr test AUC 0.5195


Hybrid_warp train AUC: 0.8637
Wall time: 11min 38s
Hybrid_warp test AUC 0.8597


Hybrid_warp-kos train AUC: 0.8624
Wall time: 11min 18s
Hybrid_warp-kos test AUC 0.8577




In [19]:
%time item_features_T = item_features.T
print(repr(item_features_T))

Wall time: 0 ns
<18689x18504 sparse matrix of type '<class 'numpy.float32'>'
	with 3441744 stored elements in Compressed Sparse Column format>


In [54]:
print(train.get_shape)

<bound method spmatrix.get_shape of <654161x18504 sparse matrix of type '<class 'numpy.float32'>'
	with 1562599 stored elements in COOrdinate format>>


In [18]:
print(repr(item_features))

<18504x18689 sparse matrix of type '<class 'numpy.float32'>'
	with 3441744 stored elements in Compressed Sparse Row format>


#### Build hybrid model with features

In [48]:
for rate in ['adagrad', 'adadelta']:
    for i in range(10, 100, 10):
        model = LightFM(loss='logistic', no_components=i, learning_schedule=rate)
        model.fit(train)
        train_auc = auc_score(model, train_sp).mean()
        test_auc = auc_score(model, test).mean()
        print('logistic_{}_{}_train: {}'.format(rate, i, train_auc))
        print('logistic_{}_{}_test: {}'.format(rate, i, test_auc))

logistic_adagrad_10_train: 0.8777787089347839
logistic_adagrad_10_test: 0.8609833717346191
logistic_adagrad_20_train: 0.8778073787689209
logistic_adagrad_20_test: 0.8610443472862244
logistic_adagrad_30_train: 0.8777976632118225
logistic_adagrad_30_test: 0.8610637187957764
logistic_adagrad_40_train: 0.8777589797973633
logistic_adagrad_40_test: 0.8610036373138428
logistic_adagrad_50_train: 0.8778204917907715
logistic_adagrad_50_test: 0.8610793352127075
logistic_adagrad_60_train: 0.8777931332588196
logistic_adagrad_60_test: 0.8611305952072144
logistic_adagrad_70_train: 0.8777799010276794
logistic_adagrad_70_test: 0.8610657453536987
logistic_adagrad_80_train: 0.8777737617492676
logistic_adagrad_80_test: 0.8610631823539734
logistic_adagrad_90_train: 0.877755343914032
logistic_adagrad_90_test: 0.8611431121826172
logistic_adadelta_10_train: 0.8802603483200073
logistic_adadelta_10_test: 0.8604311943054199
logistic_adadelta_20_train: 0.8799888491630554
logistic_adadelta_20_test: 0.8608838319778

#### Now time to tune parameters

In [95]:
# k-OS WARP: k-th order statistic loss.
'''
'''

model_kWARP = LightFM(loss='warp-kos')
%time model_kWARP.fit(train_sp)

print('Collaborative filtering train AUC: %.4f' % auc_score(model_kWARP, train_sp).mean())
print('Collaborative filtering test AUC: %.4f' % auc_score(model_kWARP, test).mean())

Wall time: 9.91 s
Collaborative filtering train AUC: 0.8880
Collaborative filtering test AUC: 0.8641


In [92]:
# WARP: Weighted Approximate-Rank Pairwise
'''

'''

model_WARP = LightFM(loss='warp')
%time model_WARP.fit(train_sp)

print('Collaborative filtering train AUC: %.4f' % auc_score(model_WARP, train_sp).mean())
print('Collaborative filtering test AUC: %.4f' % auc_score(model_WARP, test).mean())

Wall time: 7.98 s
Collaborative filtering train AUC: 0.8844
Collaborative filtering test AUC: 0.8714


In [91]:
# BPR: Bayesian Personalised Ranking

model_BPR = LightFM(loss='bpr')
%time model_BPR.fit(train_sp)

print('Collaborative filtering train AUC: %.4f' % auc_score(model_BPR, train_sp).mean())
print('Collaborative filtering test AUC: %.4f' % auc_score(model_BPR, test_sp).mean())

Wall time: 8.41 s
Collaborative filtering train AUC: 0.5127
Collaborative filtering test AUC: 0.5099


In [45]:
# loop around different loss functions with SAMPLE

'''
Logistic:
Useful when both positive and negative interactions are present.

BPR: Bayesian Personalised Ranking:
Maximises the prediction difference between a positive example and a randomly chosen negative example

WARP: Weighted Approximate-Rank Pairwise:
Maximises the rank of positive examples by repeatedly sampling negative examples until
rank violation is found. Useful when only positive ineractions are present and optimising
the top of the reommendation list is desired.

k-OS WARP: k-th order statistic loss:
A modification to WARP that uses k-th positive example for any given user as a basis for pairwise updates
'''

loss_func = ['logistic', 'bpr', 'warp', 'warp-kos']

for loss in loss_func:
    model = LightFM(loss=loss)
    model.fit(train_sp)
    
    %time print(f'CF_{loss} train AUC: {auc_score(model, train_sp).mean():.4f}')
    %time print(f'CF_{loss} test AUC {auc_score(model, test_sp).mean():.4f}')
    print('\n')

CF_logistic train AUC: 1.0000
Wall time: 2min 14s
CF_logistic test AUC 0.9999
Wall time: 54.6 s


CF_bpr train AUC: 0.6816
Wall time: 1min 28s
CF_bpr test AUC 0.0827
Wall time: 56.6 s


CF_warp train AUC: 1.0000
Wall time: 1min 12s
CF_warp test AUC 0.9999
Wall time: 55.6 s


CF_warp-kos train AUC: 1.0000
Wall time: 1min 20s
CF_warp-kos test AUC 0.9999
Wall time: 54.2 s




In [90]:
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k

train_auc = auc_score(model, train_sp).mean()
print('Collaborative filtering train AUC: %.4f' % train_auc)

test_auc = auc_score(model, test_sp).mean()
print('Collaborative filtering test AUC: %.4f' % test_auc)

print('Train precision: %.4f' % precision_at_k(model, train_sp).mean())
print('Test precision: %.4f' % precision_at_k(model, test_sp).mean())

Collaborative filtering train AUC: 0.8747
Collaborative filtering test AUC: 0.8688
Train precision: 0.0070
Test precision: 0.0043


#### Evaluation

In [28]:
# model setup
%time model = LightFM()
%time model.fit(train_sp)

Wall time: 0 ns
Wall time: 29.8 ms


In [49]:
%time print(f'Hybrid_logistic train precision: {precision_at_k(model_hybrid_hp, train, 100).mean():.4f}')
%time print(f'Hybrid_logistic test precision: {precision_at_k(model_hybrid_hp, test, 100).mean():.4f}')

ValueError: Test interactions matrix and train interactions matrix share 1562599 interactions. This will cause incorrect evaluation, check your data split.

ValueError: Test interactions matrix and train interactions matrix share 390650 interactions. This will cause incorrect evaluation, check your data split.

In [45]:
model_hybrid_hp = LightFM(loss='logistic',
                         item_alpha=2.228e-5,
                         no_components=30)
model_hybrid_hp.fit(train, item_features=item_features.T, epochs=6, num_threads=25)

%time print(f'Hybrid_logistic train AUC: {auc_score(model_hybrid_hp, train).mean():.4f}')
%time print(f'Hybrid_logistic test AUC: {auc_score(model_hybrid_hp, test).mean():.4f}')
%time print(f'Hybrid_logistic train precision: {precision_at_k(model_hybrid_hp, train).mean():.4f}')
%time print(f'Hybrid_logistic test precision: {precision_at_k(model_hybrid_hp, test).mean():.4f}')

Hybrid_logistic train AUC: 0.8686
Wall time: 19min 4s
Hybrid_logistic test AUC: 0.8634
Wall time: 7min 8s
Hybrid_logistic train precision: 0.0073
Wall time: 18min 43s
Hybrid_logistic test precision: 0.0046
Wall time: 7min 16s


In [48]:
%time print(f'CF_logistic train precision: {precision_at_k(model_cf_hp, train, k=100).mean():.4f}')
%time print(f'CF_logistic test precision: {precision_at_k(model_cf_hp, test, k=100).mean():.4f}')

CF_logistic train precision: 0.0032
Wall time: 22min 49s
CF_logistic test precision: 0.0021
Wall time: 8min 57s


In [43]:
model_cf_hp = LightFM(loss='logistic',
                     item_alpha=2.88752e-6,
                     no_components=43,
                     learning_rate=0.06652)
model_cf_hp.fit(train, epochs=18, num_threads=25)

%time print(f'CF_logistic train AUC: {auc_score(model_cf_hp, train).mean():.4f}')
%time print(f'CF_logistic test AUC: {auc_score(model_cf_hp, test).mean():.4f}')
%time print(f'CF_logistic train precision: {precision_at_k(model_cf_hp, train).mean():.4f}')
%time print(f'CF_logistic test precision: {precision_at_k(model_cf_hp, test).mean():.4f}')

CF_logistic train AUC: 0.8696
Wall time: 23min 51s
CF_logistic test AUC: 0.8637
Wall time: 9min 4s
CF_logistic train precision: 0.0073
Wall time: 24min 33s
CF_logistic test precision: 0.0046
Wall time: 9min 32s


#### CF Logistic - Hyperparameters

In [51]:
model_hybrid_warp = LightFM(no_components=50, loss='warp')
model_hybrid_warp.fit(train, item_features=item_features.T, epochs=10, num_threads=2)

%time print(f'Hybrid_warp train AUC: {auc_score(model_hybrid_warp, train, num_threads=2).mean():.4f}')
%time print(f'Hybrid_warp test AUC: {auc_score(model_hybrid_warp, test, num_threads=2).mean():.4f}')
%time print(f'Hybrid_warp train precision: {precision_at_k(model_hybrid_warp, train, num_threads=2).mean():.4f}')
%time print(f'Hybrid_warp test precision: {precision_at_k(model_hybrid_warp, test, num_threads=2).mean():.4f}')
%time print(f'Hybrid_warp train precision: {precision_at_k(model_hybrid_warp, train, num_threads=2, k=100).mean():.4f}')
%time print(f'Hybrid_warp test precision: {precision_at_k(model_hybrid_warp, test, num_threads=2, k=100).mean():.4f}')

Hybrid_warp train AUC: 0.8663
Wall time: 27min 6s
Hybrid_warp test AUC: 0.8621
Wall time: 10min 31s
Hybrid_warp train precision: 0.0073
Wall time: 26min 45s
Hybrid_warp test precision: 0.0046
Wall time: 10min 24s
Hybrid_warp train precision: 0.0032
Wall time: 26min 42s
Hybrid_warp test precision: 0.0020
Wall time: 10min 33s


In [50]:
model_cf_warp = LightFM(no_components=50, loss='warp')
model_cf_warp.fit(train, epochs=10, num_threads=2)

%time print(f'CF_warp train AUC: {auc_score(model_cf_warp, train, num_threads=2).mean():.4f}')
%time print(f'CF_warp test AUC: {auc_score(model_cf_warp, test, num_threads=2).mean():.4f}')
%time print(f'CF_warp train precision: {precision_at_k(model_cf_warp, train, num_threads=2).mean():.4f}')
%time print(f'CF_warp test precision: {precision_at_k(model_cf_warp, test, num_threads=2).mean():.4f}')
%time print(f'CF_warp train precision: {precision_at_k(model_cf_warp, train, num_threads=2, k=100).mean():.4f}')
%time print(f'CF_warp test precision: {precision_at_k(model_cf_warp, test, num_threads=2, k=100).mean():.4f}')

CF_warp train AUC: 0.9979
Wall time: 55min 43s
CF_warp test AUC: 0.9137
Wall time: 10min 1s
CF_warp train precision: 0.1062
Wall time: 25min 14s
CF_warp test precision: 0.0077
Wall time: 10min
CF_warp train precision: 0.0200
Wall time: 25min 12s
CF_warp test precision: 0.0039
Wall time: 10min 14s


#### Actual recommendations

In [76]:
dataset.

<bound method Dataset.mapping of <lightfm.data.Dataset object at 0x0000024BA21A9BE0>>

In [55]:
help(model_hybrid_warp.predict)

Help on method predict in module lightfm.lightfm:

predict(user_ids, item_ids, item_features=None, user_features=None, num_threads=1) method of lightfm.lightfm.LightFM instance
    Compute the recommendation score for user-item pairs.
    
    For details on how to use feature matrices, see the documentation
    on the :class:`lightfm.LightFM` class.
    
    Arguments
    ---------
    
    user_ids: integer or np.int32 array of shape [n_pairs,]
         single user id or an array containing the user ids for the
         user-item pairs for which a prediction is to be computed
    item_ids: np.int32 array of shape [n_pairs,]
         an array containing the item ids for the user-item pairs for which
         a prediction is to be computed.
    user_features: np.float32 csr_matrix of shape [n_users, n_user_features], optional
         Each row contains that user's weights over features.
    item_features: np.float32 csr_matrix of shape [n_items, n_item_features], optional
         Each

In [87]:
user_id_mapping, user_feature_mapping, item_id_mapping, item_feature_mapping = dataset.mapping()

In [138]:
user_id_mapping.get('-hJ9Q0OSYqdOTVBLOMdg7A')

495702

In [100]:
business_lv = {b: val for b, val in item_id_mapping.items() if b in b_lv}
business_lv

{'iCQpiavjjPzJ5_3gPD5Ebg': 0,
 'pomGBqfbxcqPv14c3XH-ZQ': 1,
 'jtQARsP6P-LbkyjbO1qNGg': 2,
 'elqbBhBfElMNSrjFqW3now': 3,
 'zdE82PiD6wquvjYLyhOJNA': 4,
 'EAwh1OmG6t6p3nRaZOW_AA': 5,
 'atVh8viqTj-sqDJ35tAYVg': 6,
 'faPVqws-x-5k2CQKDNtHxw': 39,
 '6tSvz_21BMo3a4GaItwa0g': 40,
 '-ed0Yc9on37RoIoG2ZgxBA': 41,
 'XiN6fI8I3Mzg2nPRJ9ukRQ': 42,
 'BEtgRzNeXGAf0uQ-HuSyfA': 43,
 '7wHLFohwCw8l6WS-feLjeg': 44,
 'ixAh9crILnJ9tM8LhWFhkw': 45,
 'aQ222ydz_GSRZV66xNt4kQ': 46,
 'VmspRdKauEargr8YX1oL-Q': 47,
 'xqbvqZHNyj2qExHdizzd0w': 48,
 'JPfi__QJAaRzmfh5aOyFEw': 49,
 'ruWTngdiC68O9la27hvvHw': 83,
 '7KkgMcbVaetryW1wwpzvvA': 84,
 'bvGdqJ-SeGdIHX-HEabwlg': 85,
 '9tLCuUmw3Tlco-tKPGkIWQ': 86,
 '7UFDAX4wLi6ux5otguYldA': 87,
 '3kdSl5mo9dWC4clrQjEDGg': 88,
 'f4x1YBxkLrZg652xt2KR5g': 89,
 'DkYS3arLOhA8si5uUEmHOw': 90,
 'fL-b760btOaGa85OJ9ut3w': 91,
 'eBj_YyJU5jVu6tbZCkdtDA': 116,
 'DB9BRdu7is0AK1DvcuKlcw': 117,
 'K7lWdNUhCbcnEvI0NhGewg': 118,
 'wuQDMDlqM17jQNo0lYQZ7g': 122,
 'gqdqnFg2lYeSyS20q2ZK_Q': 123,
 'h5qBxa_L

In [129]:
model_cf_warp.predict(user_ids=[43559], item_ids=[i for i in business_lv.values()], num_threads=2)

array([ 1.96271384,  0.25593841, -0.6797145 , ..., -1.17264915,
       -0.99065953, -1.32866204])

In [159]:
list(r.business_id[(r.user_id == '-hJ9Q0OSYqdOTVBLOMdg7A') & (r.city == 'Las Vegas') & (r.sentiment == 1)])

['NG2mL-wQQfBdsXsyUbw8pA',
 'pomGBqfbxcqPv14c3XH-ZQ',
 'qJEtXdrOQeo9py0Wqr6LKQ',
 'UDE4VL0k8Zrko7RkJ4agGQ',
 'FNe5PPA9pyj8FjcDefCBpg',
 '2iTsRqUsPGRH1li1WVRvKQ']

In [165]:
b_list = list(r.business_id[(r.user_id == '-hJ9Q0OSYqdOTVBLOMdg7A') & (r.city == 'Las Vegas') & (r.sentiment == 1)])
b_map = [item_id_mapping.get(i) for i in b_list]
score = model_cf_warp.predict(user_ids=[495702], item_ids=b_map, num_threads=2)
reco = dict(zip(b_list, score))

In [168]:
sorted(reco.values())

[1.0769048929214478,
 1.2248644828796387,
 1.4924488067626953,
 1.9236305952072144,
 2.0735278129577637,
 2.4990010261535645]

In [173]:
sorted(reco.keys(), key=lambda x: x[1], reverse=True)

['pomGBqfbxcqPv14c3XH-ZQ',
 '2iTsRqUsPGRH1li1WVRvKQ',
 'FNe5PPA9pyj8FjcDefCBpg',
 'qJEtXdrOQeo9py0Wqr6LKQ',
 'NG2mL-wQQfBdsXsyUbw8pA',
 'UDE4VL0k8Zrko7RkJ4agGQ']

In [221]:
def make_recommendation(model, user_ids, city='Las Vegas', top_n=3):
    """
    """
    business_map = [item_id_mapping.get(i) for i in b.index[b.city == city]]
    
    for user_id in user_ids:
        known_positives = list(r.business_id[(r.user_id == user_id) & (r.city == city) & (r.sentiment == 1)])
        
        user_map = user_id_mapping.get(user_id)
        
        scores = model.predict(user_ids=user_map, item_ids=business_map, num_threads=2)
        recommendation = dict(zip(known_positives, scores))
        top_items = sorted(recommendation.keys(), key=lambda x: x[1], reverse=True)[:10]

        print(f"User {user_id}")
        print("     Known positives:")
        print(b[business_features_reco][b.index.isin(known_positives[:top_n])])

        print("     Recommended:")
        print(b[business_features_reco][b.index.isin(top_items[:top_n])])

In [219]:
make_recommendation(model_cf_warp, ['-hJ9Q0OSYqdOTVBLOMdg7A'], city='Las Vegas', top_n=3)

User -hJ9Q0OSYqdOTVBLOMdg7A
     Known positives:
                                             name  stars  price
business_id                                                    
NG2mL-wQQfBdsXsyUbw8pA  Capriotti's Sandwich Shop    3.0    2.0
pomGBqfbxcqPv14c3XH-ZQ   Leticia's Mexican Cocina    4.0    2.0
qJEtXdrOQeo9py0Wqr6LKQ     Blaze Fast Fired Pizza    4.0    1.0
     Recommended:
                                            name  stars  price
business_id                                                   
2iTsRqUsPGRH1li1WVRvKQ            Carson Kitchen    4.5    2.0
FNe5PPA9pyj8FjcDefCBpg                Weera Thai    4.5    2.0
pomGBqfbxcqPv14c3XH-ZQ  Leticia's Mexican Cocina    4.0    2.0


#### WARP CF & Hybrid

In [47]:
%time print(f'Hybrid_logistic train precision: {precision_at_k(model_hybrid, train, k=100).mean():.4f}')
%time print(f'Hybrid_logistic test precision: {precision_at_k(model_hybrid, test, k=100).mean():.4f}')

Hybrid_logistic train precision: 0.0032
Wall time: 11min 34s
Hybrid_logistic test precision: 0.0020
Wall time: 4min 23s


In [42]:
model_hybrid = LightFM()
model_hybrid.fit(train, item_features=item_features.T)

%time print(f'Hybrid_logistic train AUC: {auc_score(model_hybrid, train).mean():.4f}')
%time print(f'Hybrid_logistic test AUC: {auc_score(model_hybrid, test).mean():.4f}')
%time print(f'Hybrid_logistic train precision: {precision_at_k(model_hybrid, train).mean():.4f}')
%time print(f'Hybrid_logistic test precision: {precision_at_k(model_hybrid, test).mean():.4f}')

Hybrid_logistic train AUC: 0.8570
Wall time: 12min 3s
Hybrid_logistic test AUC: 0.8528
Wall time: 4min 33s
Hybrid_logistic train precision: 0.0073
Wall time: 11min 48s
Hybrid_logistic test precision: 0.0046
Wall time: 4min 25s


#### Hybrid Logistic

In [46]:
%time print(f'CF_logistic train precision: {precision_at_k(model_cf, train, k=100).mean():.4f}')
%time print(f'CF_logistic test precision: {precision_at_k(model_cf, test, k=100).mean():.4f}')

CF_logistic train precision: 0.0032
Wall time: 11min 19s
CF_logistic test precision: 0.0020
Wall time: 4min 38s


In [40]:
model_cf = LightFM()
model_cf.fit(train)

%time print(f'CF_logistic train AUC: {auc_score(model_cf, train).mean():.4f}')
%time print(f'CF_logistic test AUC: {auc_score(model_cf, test).mean():.4f}')
%time print(f'CF_logistic train precision: {precision_at_k(model_cf, train).mean():.4f}')
%time print(f'CF_logistic test precision: {precision_at_k(model_cf, test).mean():.4f}')

CF_logistic train AUC: 0.8672
Wall time: 12min 55s
CF_logistic test AUC: 0.8626
Wall time: 7min 50s
CF_logistic train precision: 0.0074
Wall time: 17min 6s
CF_logistic test precision: 0.0046
Wall time: 4min 43s
